In [1]:
# breadth-first search will be used to solve graph problems
from enum import Enum, IntEnum
from typing import (
    Tuple,
    List,
    TypeVar,
    Iterable,
    Sequence,
    Generic,
    Callable,
    Set,
    Deque,
    Dict,
    Any,
    Optional,
    NamedTuple
)
from typing_extensions import Protocol
import random
import sys
import bisect
from heapq import heappush, heappop
from math import sqrt

T = TypeVar('T')

class Node(Generic[T]):
    def __init__(
        self,
        state: T,
        parent, # Optional[Node]
        cost: float = 0.0,
        heuristic: float = 0.0
    ) -> None:
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic
        
    def __lt__(self, other) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
    
class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
        
    def pop(self) -> T:
        return self._container.popleft()
    
    def __repr__(self) -> str:
        return repr(self._container)
    
class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []

    @property
    def empty(self) -> bool:
        return not self._container

    def push(self, item: T) -> None:
        heappush(self._container, item)

    def pop(self) -> T:
        return heappop(self._container)

    def __repr__(self) -> str:
        return repr(self._container)
    
def bfs(
    initial: T,
    goal_test: Callable[[T], bool],
    successors: Callable[[T], List[T]]
) -> Optional[Node[T]]:
    
    # where we have yet to go
    frontier: Queue[Node[T]] = Queue()
    frontier.push(Node(initial, None))
    
    # where we have been
    explored: Set[T] = {initial}
        
    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
            
        # if we find the goal we are done
        if goal_test(current_state):
            return current_node
        
        # check were we can go and have not explored
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    return

def node_to_path(node: Node[T]) -> List[T]:

    path: List[T] = [node.state]
    
    # work backwards from end to head of linked list
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
        
    path.reverse()
    return path

# Unweighted Graph

In [2]:
exec('from __future__ import annotations')
from dataclasses import dataclass
from typing import TypeVar, Generic, List, Optional

In [3]:
@dataclass
class Edge:
    u: int  # from vertex
    v: int  # to vertex
        
    def __str__(self) -> str:
        return f'{self.u} -> {self.v}'

def rev(self) -> Edge:
    return Edge(self.v, self.u)

Edge.reversed = rev

In [4]:
# type for vertices
V = TypeVar('V')

In [5]:
class Graph(Generic[V]):
    def __init__(self, vertices: List[V]) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[Edge]] = [[] for _ in vertices]
    
    @property
    def vertex_count(self) -> int:
        return len(self._vertices)
    
    @property
    def edge_count(self) -> int:
        return sum(map(len, self._edges))
    
    # add vertex to graph and return index
    def add_vertex(self, vertex: V) -> int:
        self._vertices.append(vertex)
        self._edges.append([])
        return self.vertex_count - 1
    
    # undirected graph: append edges in both directions
    def add_edge(self, edge: Edge) -> None:
        self._edges[edge.u].append(edge)
        self._edges[edge.v].append(edge.reversed())
        
    # add edge using using vertex indices (convenience method)
    def add_edge_by_indices(self, u: int, v: int) -> None:
        edge: Edge = Edge(u, v)
        self.add_edge(edge)
        
    # add an edge by looking up vertex indices (convenience method)
    def add_edge_by_vertices(self, first: V, second: V) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v)
        
    # find vertex at specific index
    def vertex_at(self, index: int) -> V:
        return self._vertices[index]
    
    # find the index of specific vertex
    def index_of(self, vertex: V) -> int:
        return self._vertices.index(vertex)
    
    # find the vertices that a vertex at some index is connected to
    def neighbors_for_index(self, index: int) -> List[V]:
        return list(map(self.vertex_at, [e.v for e in self._edges[index]]))
    
    # lookup a vertice's index and find its neighbors (convenience method)
    def neighbors_for_vertex(self, vertex: V) -> List[V]:
        return self.neighbors_for_index(self.index_of(vertex))
    
    # return all the edges associated with a vertex at some index
    def edges_for_index(self, index: int) -> List[Edge]:
        return self._edges[index]
    
    # look up the index of a vertex and return its edges (convenience method)
    def edges_for_vertex(self, vertex: V) -> List[Edge]:
        return self.edges_for_index(self.index_of(vertex))
    
    # pretty-print
    def __str__(self) -> str:
        desc = ''
        for i in range(self.vertex_count):
            desc += f'{self.vertex_at(i)} -> {self.neighbors_for_index(i)}\n'
        return desc

In [6]:
city_graph: Graph[str] = Graph([
    'Seattle',
    'San Francisco',
    'Los Angeles',
    'Riverside',
    'Phoenix',
    'Chicago',
    'Boston',
    'New York',
    'Atlanta',
    'Miami',
    'Dallas',
    'Houston',
    'Detroit',
    'Philadelphia',
    'Washington'
])
    
city_graph.add_edge_by_vertices('Seattle', 'Chicago')
city_graph.add_edge_by_vertices('Seattle', 'San Francisco')
city_graph.add_edge_by_vertices('San Francisco', 'Riverside')
city_graph.add_edge_by_vertices('San Francisco', 'Los Angeles')
city_graph.add_edge_by_vertices('Los Angeles', 'Riverside')
city_graph.add_edge_by_vertices('Los Angeles', 'Phoenix')
city_graph.add_edge_by_vertices('Riverside', 'Phoenix')
city_graph.add_edge_by_vertices('Riverside', 'Chicago')
city_graph.add_edge_by_vertices('Phoenix', 'Dallas')
city_graph.add_edge_by_vertices('Phoenix', 'Houston')
city_graph.add_edge_by_vertices('Dallas', 'Chicago')
city_graph.add_edge_by_vertices('Dallas', 'Houston')
city_graph.add_edge_by_vertices('Dallas', 'Atlanta')
city_graph.add_edge_by_vertices('Houston', 'Atlanta')
city_graph.add_edge_by_vertices('Houston', 'Miami')
city_graph.add_edge_by_vertices('Atlanta', 'Chicago')
city_graph.add_edge_by_vertices('Atlanta', 'Washington')
city_graph.add_edge_by_vertices('Atlanta', 'Miami')
city_graph.add_edge_by_vertices('Miami', 'Washington')
city_graph.add_edge_by_vertices('Chicago', 'Detroit')
city_graph.add_edge_by_vertices('Detroit', 'Boston')
city_graph.add_edge_by_vertices('Detroit', 'Washington')
city_graph.add_edge_by_vertices('Detroit', 'New York')
city_graph.add_edge_by_vertices('Boston', 'New York')
city_graph.add_edge_by_vertices('New York', 'Philadelphia')
city_graph.add_edge_by_vertices('Philadelphia', 'Washington')
print(city_graph)

Seattle -> ['Chicago', 'San Francisco']
San Francisco -> ['Seattle', 'Riverside', 'Los Angeles']
Los Angeles -> ['San Francisco', 'Riverside', 'Phoenix']
Riverside -> ['San Francisco', 'Los Angeles', 'Phoenix', 'Chicago']
Phoenix -> ['Los Angeles', 'Riverside', 'Dallas', 'Houston']
Chicago -> ['Seattle', 'Riverside', 'Dallas', 'Atlanta', 'Detroit']
Boston -> ['Detroit', 'New York']
New York -> ['Detroit', 'Boston', 'Philadelphia']
Atlanta -> ['Dallas', 'Houston', 'Chicago', 'Washington', 'Miami']
Miami -> ['Houston', 'Atlanta', 'Washington']
Dallas -> ['Phoenix', 'Chicago', 'Houston', 'Atlanta']
Houston -> ['Phoenix', 'Dallas', 'Atlanta', 'Miami']
Detroit -> ['Chicago', 'Boston', 'Washington', 'New York']
Philadelphia -> ['New York', 'Washington']
Washington -> ['Atlanta', 'Miami', 'Detroit', 'Philadelphia']



In [7]:
# in an unweighted graph, BFS will find the shortest path between any two vertices
result: Optional[Node[V]] = bfs('Seattle', lambda x: x == 'New York', city_graph.neighbors_for_vertex)
if not result:
    print('No result found.')
else:
    path: List[V] = node_to_path(result)
    print('Path from Boston to Miami:')
    print(path)

Path from Boston to Miami:
['Seattle', 'Chicago', 'Detroit', 'New York']


# Weighted Graph

In [8]:
@dataclass
class WeightedEdge(Edge):
    weight: float
        
    def __lt__(self, other) -> bool:
        return self.weight < other.weight
    
    def __str__(self) -> str:
        return f'{self.u} {self.weight}> {self.v}'
        
def rev(self) -> WeightedEdge:
    return WeightedEdge(self.v, self.u, self.weight)

WeightedEdge.reversed = rev

In [9]:
class WeightedGraph(Generic[V], Graph[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[WeightedEdge]] = [[] for _ in vertices]
        
    def add_edge_by_indices(self, u: int, v: int, weight: float) -> None:
        edge: WeightedEdge = WeightedEdge(u, v, weight)
        self.add_edge(edge)
        
    def add_edge_by_vertices(self, first: V, second: V, weight: float) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v, weight)
        
    def neighbors_for_index_with_weights(self, index: int) -> List[Tuple[V, float]]:
        distance_tuples: List[Tuple[V, float]] = []
        for edge in self.edges_for_index(index):
            distance_tuples.append((self.vertex_at(edge.v), edge.weight))
        return distance_tuples
    
    def __str__(self) -> str:
        desc = ''
        for i in range(self.vertex_count):
            desc += f'{self.vertex_at(i)} -> {self.neighbors_for_index_with_weights(i)}\n'
        return desc

In [10]:
city_graph2: WeightedGraph[str] = WeightedGraph([
    'Seattle',
    'San Francisco',
    'Los Angeles',
    'Riverside',
    'Phoenix',
    'Chicago',
    'Boston',
    'New York',
    'Atlanta',
    'Miami',
    'Dallas',
    'Houston',
    'Detroit',
    'Philadelphia',
    'Washington'
])
    
city_graph2.add_edge_by_vertices('Seattle', 'Chicago', 1737)
city_graph2.add_edge_by_vertices('Seattle', 'San Francisco', 678)
city_graph2.add_edge_by_vertices('San Francisco', 'Riverside', 386)
city_graph2.add_edge_by_vertices('San Francisco', 'Los Angeles', 348)
city_graph2.add_edge_by_vertices('Los Angeles', 'Riverside', 50)
city_graph2.add_edge_by_vertices('Los Angeles', 'Phoenix', 357)
city_graph2.add_edge_by_vertices('Riverside', 'Phoenix', 307)
city_graph2.add_edge_by_vertices('Riverside', 'Chicago', 1704)
city_graph2.add_edge_by_vertices('Phoenix', 'Dallas', 887)
city_graph2.add_edge_by_vertices('Phoenix', 'Houston', 1015)
city_graph2.add_edge_by_vertices('Dallas', 'Chicago', 805)
city_graph2.add_edge_by_vertices('Dallas', 'Houston', 225)
city_graph2.add_edge_by_vertices('Dallas', 'Atlanta', 721)
city_graph2.add_edge_by_vertices('Houston', 'Atlanta', 702)
city_graph2.add_edge_by_vertices('Houston', 'Miami', 968)
city_graph2.add_edge_by_vertices('Atlanta', 'Chicago', 588)
city_graph2.add_edge_by_vertices('Atlanta', 'Washington', 543)
city_graph2.add_edge_by_vertices('Atlanta', 'Miami', 604)
city_graph2.add_edge_by_vertices('Miami', 'Washington', 923)
city_graph2.add_edge_by_vertices('Chicago', 'Detroit', 238)
city_graph2.add_edge_by_vertices('Detroit', 'Boston', 613)
city_graph2.add_edge_by_vertices('Detroit', 'Washington', 396)
city_graph2.add_edge_by_vertices('Detroit', 'New York', 482)
city_graph2.add_edge_by_vertices('Boston', 'New York', 190)
city_graph2.add_edge_by_vertices('New York', 'Philadelphia', 81)
city_graph2.add_edge_by_vertices('Philadelphia', 'Washington', 123)
print(city_graph2)

Seattle -> [('Chicago', 1737), ('San Francisco', 678)]
San Francisco -> [('Seattle', 678), ('Riverside', 386), ('Los Angeles', 348)]
Los Angeles -> [('San Francisco', 348), ('Riverside', 50), ('Phoenix', 357)]
Riverside -> [('San Francisco', 386), ('Los Angeles', 50), ('Phoenix', 307), ('Chicago', 1704)]
Phoenix -> [('Los Angeles', 357), ('Riverside', 307), ('Dallas', 887), ('Houston', 1015)]
Chicago -> [('Seattle', 1737), ('Riverside', 1704), ('Dallas', 805), ('Atlanta', 588), ('Detroit', 238)]
Boston -> [('Detroit', 613), ('New York', 190)]
New York -> [('Detroit', 482), ('Boston', 190), ('Philadelphia', 81)]
Atlanta -> [('Dallas', 721), ('Houston', 702), ('Chicago', 588), ('Washington', 543), ('Miami', 604)]
Miami -> [('Houston', 968), ('Atlanta', 604), ('Washington', 923)]
Dallas -> [('Phoenix', 887), ('Chicago', 805), ('Houston', 225), ('Atlanta', 721)]
Houston -> [('Phoenix', 1015), ('Dallas', 225), ('Atlanta', 702), ('Miami', 968)]
Detroit -> [('Chicago', 238), ('Boston', 613), 

In [11]:
WeightedPath = List[WeightedEdge]

def total_weight(path: WeightedPath) -> float:
    return sum([e.weight for e in path])

## Jarnik's (Prim's) algorithm
This algorithm finds the minimum spanning tree (MST) from a directed weighted graph.
The graph is split into two parts: vertices in the still-being-assembled MST and vertices not in it.
There are three steps:
* start by picking an arbitrary vertex to include in the MST
* find lowest-weighted edge to connect vertices not yet in the MST to the MST
* add the vertex at the edd of the minimum edge to the MST (thanks to the priority queue)
* repeat steps 2 and 3 until every vertex is in the MST

Jarnik's algorithm is considered *greedy* because it always selects the lowest weighted edge.

Notes: Jarnik's algorithm will not always find the best path in a directed graph and will not work in a graph which is not connected.


In [12]:
def mst(graph: WeightedGraph, start: int = 0) -> Optional[WeightedPath]:
    if start > (graph.vertex_count - 1) or start < 0:
        return
    
    result: WeightedPath = []
    queue: PriorityQueue[WeightedEdge] = PriorityQueue()
    visited: List[bool] = [False] * graph.vertex_count
    
    def visit(index: int) -> None:
        visited[index] = True
        for edge in graph.edges_for_index(index):
            if not visited[edge.v]:
                queue.push(edge)
                
    visit(start)
    
    while not queue.empty:
        edge = queue.pop()
        if visited[edge.v]:
            continue
        result.append(edge)  # smallest weight thanks to priority queue
        visit(edge.v)
        
    return result

In [17]:
def print_weighted_path(graph: WeightedGraph, path: WeightedPath) -> str:
    for edge in path:
        print(f'{graph.vertex_at(edge.u)} {edge.weight}> {graph.vertex_at(edge.v)}')
    print(f'Total weight: {total_weight(path)}')

In [18]:
result2 = mst(city_graph2)
if not result2:
    print('Path not found.')
else:
    print_weighted_path(city_graph2, result2)

Seattle 678> San Francisco
San Francisco 348> Los Angeles
Los Angeles 50> Riverside
Riverside 307> Phoenix
Phoenix 887> Dallas
Dallas 225> Houston
Houston 702> Atlanta
Atlanta 543> Washington
Washington 123> Philadelphia
Philadelphia 81> New York
New York 190> Boston
Washington 396> Detroit
Detroit 238> Chicago
Atlanta 604> Miami
Total weight: 5372


## Dijkstra's algorithm
We want to find the shortest paths in a weighted graph: what is the shortest path (total edge weight) from some vertex to any other vertex in the weighted graph?

Algorithm steps:
1. add starting vertex to the priority queue
2. pop the closest vertex from the priority queue -> current vertex
3. look at all vertices connect to current vertex: if they were not visited or if the edge provides a new shortest path to them, then for each vertex record distance from start, record the edge which produces this distance and add new vertex to priority queue
4. repeat steps 2 and 3 until queue is empty
5. return shortest path to each vertex from the starting vertex and the path to get to each of them

Like Jarnik's algorithm, Dijstra's algorithm is greedy and they use similar code.

Jarnik's algorithm is similar to the A* algorithm, except that the A* algorithm has a heuristic and finds a single destination.

In [15]:
@dataclass
class DijkstraNode:
    vertex: int
    distance: float
    
    def __lt__(self, other) -> bool:
        return self.distance < other.distance
    
    def __eq__(self, other) -> bool:
        return self.distance == other.distance


def dijkstra(graph: WeightedGraph[V], root: V) -> Tuple[List[Optional[float]], Dict[int, WeightedEdge]]:
    first: int = graph.index_of(root)  # starting index
    distances: List[Optional[float]] = [None] * graph.vertex_count  # distances are unknown at first
    distances[first] = 0  # root is zero away from the root
    path_dict: Dict[int, WeightedEdge] = {}  # how we got to each vertex
    queue: PriorityQueue[DijkstraNode] = PriorityQueue()
    queue.push(DijkstraNode(first, 0))
    
    while not queue.empty:
        u: int = queue.pop().vertex  # get closest vertex
        dist_u: float = distances[u]  # should already have seen it
        
        # look at every edge/vertex from the current vertex
        for edge in graph.edges_for_index(u):
            # previously recorded distance for this vertex
            dist_v: float = distances[edge.v]
            if dist_v is None or dist_v > dist_u + edge.weight:
                # update shortest distance to this vertex
                distances[edge.v] = dist_u + edge.weight
                # update edge on the shortest path to this vertex
                path_dict[edge.v] = edge
                # explore it soon
                queue.push(DijkstraNode(edge.v, dist_u + edge.weight))
                
    return distances, path_dict


def distance_array_to_vertex_dict(
    graph: WeightedGraph[V],
    distances: List[Optional[float]]
) -> Dict[V, Optional[float]]:
    """Helper function to get easier access to Dijkstra's result"""
    distance_dict: Dict[V, Optional[float]] = {}
    for i in range(len(distances)):
        distance_dict[graph.vertex_at(i)] = distances[i]
    return distance_dict


def path_dict_to_path(
    start: int,
    end: int,
    path_dict: Dict[int, WeightedEdge]
) -> WeightedPath:
    """
    Take a dictionary of edges to reach each node and return
    a list of edges that goes from start to end.
    """
    if len(path_dict) == 0:
        return []
    edge_path: WeightedPath = []
    e: WeightedEdge = path_dict[end]
    edge_path.append(e)
    while e.u != start:
        e = path_dict[e.u]
        edge_path.append(e)
    return list(reversed(edge_path))

In [19]:
distances, path_dict = dijkstra(city_graph2, 'Los Angeles')
name_distance: Dict[str, Optional[int]] = distance_array_to_vertex_dict(city_graph2, distances)
print('Distances from Los Angeles:')
for k, v in name_distance.items():
    print(f'{k}: {v}')
print()

print('Shortest path from Los Angeles to Boston:')
path: WeightedPath = path_dict_to_path(
    city_graph2.index_of('Los Angeles'),
    city_graph2.index_of('Boston'),
    path_dict
)
print_weighted_path(city_graph2, path)

Distances from Los Angeles:
Seattle: 1026
San Francisco: 348
Los Angeles: 0
Riverside: 50
Phoenix: 357
Chicago: 1754
Boston: 2605
New York: 2474
Atlanta: 1965
Miami: 2340
Dallas: 1244
Houston: 1372
Detroit: 1992
Philadelphia: 2511
Washington: 2388

Shortest path from Los Angeles to Boston:
Los Angeles 50> Riverside
Riverside 1704> Chicago
Chicago 238> Detroit
Detroit 613> Boston
Total weight: 2605
